In [ ]:
import ohawf
from apiclient.discovery import build

In [ ]:
creds = ohawf.get()

In [ ]:
service = build('analytics', 'v3', credentials=creds)

In [ ]:
accounts = service.management().accounts().list().execute()

In [ ]:
for adict in accounts["items"]:
    print(f"Name: {adict['name']}")
    print(f"ID: {adict['id']}")
    print()